# Wine Services WebScraping
Jack d'Entremont

Welcome!  First time users, follow the appropriate instructions in the attached documents to view Wine Services data.


The following code imports the necessary extensions for the program to work.  First time users, be sure to follow all instructions in the attached documents prior to running the code.  After running the code, when prompted, select 'Python Environments' and the most up to date option.  To run the code, click anywhere in the block, and press Shift and Enter at the same time.

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import *
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import *
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import helperFunctions
import numpy as np
import pandas as pd
from IPython.display import display
import openpyxl
import re

Now that you have imported all extensions, it is time to tell the program where your driver is located.

In the code below, please enter the path to the web driver on your system.  If the driver is in your downloads, your path might look like this:

'/Users/john.smith/Downloads/chromedriver_mac64'

Once you have entered the path, run the code.

NOTE:  Running this code opens an internet browser.  DO NOT interact with this window.  Ensure it is open in full screen, then leave it running in the background, and return to this program.  Run the code below by clicking anywhere in the box and pressing shift and enter at the same time.

In [ ]:
driver_path = '/Users/jack.dentremont/Downloads/chromedriver-mac-arm64/chromedriver'
service = ChromeService(driver_path)
driver = webdriver.Chrome(service=service)
driver.get('https://myapp.wine-services.com/dashboard')

Run the code in the following box to navigate to the correct webpage.  Run by clicking anywhere in the box, and pressing shift+enter.

In [ ]:
email = driver.find_element(By.XPATH, '//input[@label="email"]')
email.send_keys('jack.dentremont@cbrands.com')
password = driver.find_element(By.XPATH, '//input[@label="Mot de passe"]')
password.send_keys('wG88K86Biz')
button = driver.find_element(By.XPATH, '//div[@class="loginSubmitContainer"]')
button.click()

driver.implicitly_wait(5)

accept_terms = driver.find_element(By.XPATH, '//div[@class="sc-EHOje bxzNxf"]')
accept_terms.click()

driver.implicitly_wait(5)

distribution = driver.find_element(By.XPATH, '//a[@href="/distribution"]')
distribution.click()

driver.implicitly_wait(5)

try:
    sidebar = driver.find_element(By.XPATH, '//div[@class="sc-htpNat QdHme left-sidebar"]')
    sidebar_width = sidebar.size['width']
    offset_x = sidebar_width + 100
    actions = ActionChains(driver)
    actions.move_by_offset(offset_x, sidebar_width)
    actions.perform()
except NoSuchElementException:
    driver.implicitly_wait(5)

The following code will prompt you for your choice of Market.  Run the code by clicking anywhere in the box and pressing shift+enter.

In [ ]:
MARKETS = set()
try:
    sidebar = driver.find_element(By.XPATH, '//div[@class="sc-htpNat QdHme left-sidebar"]')
    sidebar_width = sidebar.size['width']
    offset_x = sidebar_width + 100
    actions = ActionChains(driver)
    actions.move_by_offset(offset_x, sidebar_width)
    actions.perform()
except NoSuchElementException:
    driver.implicitly_wait(5)
market = driver.find_element(By.XPATH, '//div[@class="sc-bdVaJa keKdli"]')
market.click()
lyst = driver.find_elements(By.XPATH, '//p[@class="countryLabelCity"]')
for element in lyst:
    country = element.text
    MARKETS.add(country)
for index, item in enumerate(MARKETS, 1):
    print(f"{index}. {item}")
mkt = input("\nPlease enter the number of your desired market: ")
if not mkt.isdigit() or not (int(mkt) > 0 and int(mkt) <= len(MARKETS)):
    print("\nInvalid entry.  Try again.")
name = driver.find_element(By.XPATH, '//input[@id="react-select-2-input"]')
try:
    name.send_keys(list(MARKETS)[int(mkt) - 1])
    name.send_keys(Keys.RETURN)
except ValueError:
    print("Invalid entry. Please select Clear All Outputs from the top of the screen, and start over.")

Some markets allow the choice between Restaurants and Wine Shops.  Run the code below to enter your choice, if prompted.

In [ ]:
driver.implicitly_wait(5)

try:
    unclickable = driver.find_element(By.XPATH, '//div[@class="sc-iyvyFf cCDKOY"]')
    print('\n' + list(MARKETS)[int(mkt) - 1] + " only has data available for Restaurants.")
except NoSuchElementException:
    clickables = driver.find_elements(By.XPATH, '//div[@class="sc-iyvyFf jcqYhu"]')
    print("\nWhich network would you like to view?\n1. Restaurants\n2. Wine Shops\n")
    ntk = input("Please enter your choice (1 or 2): ")
    if not ntk.isdigit() or not (int(ntk) == 1 or int(ntk) == 2):
        print("Invalid entry.  Please run this block of code again.")
    if int(ntk) == 1:
        clickables[0].click()
    if int(ntk) == 2:
        clickables[1].click()

Now it is time to select the brands you would like to view.  Run the following code by clicking anywhere in the box, pressing Shift and Enter at the same time.

In [ ]:
driver.implicitly_wait(5)
clear = driver.find_element(By.XPATH, '//*[@id="root"]/div/div/div[3]/div/div[3]/div/div/div/div[4]/div[2]/div/div[2]/div[1]')
clear.click()
BRANDS = set()
brand = driver.find_element(By.XPATH, '//*[@id="root"]/div/div/div[3]/div/div[3]/div/div/div/div[4]/div[2]/div/div[1]/div[1]')
brand.click()
lyst = driver.find_elements(By.XPATH, '//*[@id="root"]/div/div/div[3]/div/div[3]/div/div/div/div[4]/div[2]/div[2]')
for element in lyst:
    brnd = element.text
    BRANDS.add(brnd)
BRANDS_SET = next(iter(BRANDS))
BRANDS_LYST = BRANDS_SET.split('\n')
if 'All' in BRANDS_LYST:
    BRANDS_LYST.remove('All')
print("\nThe brands with data available are:")
for index, item in enumerate(BRANDS_LYST, 1):
    print(f"{index}. {item}")
brnds = []
while True:
    fdbck = input("\nPlease enter, one at a time, the number of each brand you would like to view.  Press Enter after each choice.  Type 'Done' when finished, or 'All' to select all.")
    if fdbck == "Done":
        break
    elif fdbck == "All":
        brnds = list(range(1, len(BRANDS_LYST) + 1))
        break
    else:
        brnds.append(fdbck)
answr = driver.find_element(By.XPATH, '//input[@id="react-select-4-input"]')
for index in brnds:
    answr.send_keys(BRANDS_LYST[int(index) - 1])
    answr.send_keys(Keys.RETURN)

Now that you have selected the brands of interest, you can select the date of the survey you would like to view.  Some markets may allow for the 'Current view' option, which would be the most up to date data.  Run the following code by clicking anywhere in the box and pressing shift+enter.

In [ ]:
driver.implicitly_wait(5)

DATES = set()
date_choice = driver.find_element(By.XPATH, '(//div[@class="css-1k3a2v7"])[2]')
date_choice.click()
dates_lyst = []
try:
    for item in driver.find_elements(By.XPATH, '//*[@id="root"]/div/div/div[3]/div/div[3]/div/div/div/div[3]/div[2]/div[2]'):
        dates_lyst.append(item.text)
except NoSuchElementException:
    driver.implicitly_wait(5)
dates_lyst = next(iter(dates_lyst))
dates_lyst = dates_lyst.split('\n')
if len(dates_lyst) == 1:
    print("\nThere is just one survey available for this market: " + "".join(dates_lyst[0] + "."))
else:
    print("The Survey Dates available are:")
    for index, item in enumerate(dates_lyst, 1):
        print(f"{index}. {item}")
    date = input("Please enter the number of the Survey you would like to select: ")
    inpt = driver.find_element(By.XPATH, '//input[@id="react-select-3-input"]')
    inpt.send_keys(dates_lyst[int(date) - 1])
    inpt.send_keys(Keys.RETURN)

Now it is time to select the competitors to compare against.  By default, Wine Services includes all available competitors in your market of choice.  By running the code below, you can select competitors you DO NOT want to see data for.  Run the code below by clicking anywhere in the box, and presing Shift and Enter at the same time.

In [ ]:
COMPETITORS = set()
nos = []
edit_button = driver.find_element(By.XPATH, '//div[@class="inputModalButton"]')
edit_button.click()
comp_lyst = driver.find_elements(By.XPATH, '//div[@class="checkboxLabel"]')
for item in comp_lyst:
    COMPETITORS.add(item.text)
if 'All' in COMPETITORS:
    COMPETITORS.remove('All')
COMPETITORS = {value for value in COMPETITORS if value} 
print("\nThe competitors for the brand(s) you selected are:")
for index, item in enumerate(COMPETITORS, 1):
    print(f"{index}. {item}")
while True:
    rmv = input("Please enter, one at a time, the number of any competitor you would like to exclude, pressing Enter after each one.  Type 'Skip' to keep all competitors, and 'Done' when finished.")
    if rmv == 'Skip' or rmv == 'Done':
        break
    elif not rmv.isdigit() or not (1 <= int(rmv) <= len(COMPETITORS)):
        print("Invalid entry.  Please run this block of code again.")
    else:
        nos.append(list(COMPETITORS)[int(rmv) - 1])
for comp in nos:
    elements = driver.find_elements(By.XPATH, '//*[@id="root"]/div/div/div[3]/div/div[3]/div/div/div/div[5]/div[2]/div[2]/div[2]/div[1]/div/p')
    for element in elements:
        if element.text == comp:
            element.find_element(By.XPATH, './preceding-sibling::*[1]').click()
            break

validate = driver.find_element(By.XPATH, '//div[@class="sc-EHOje bxzNxf"]')
validate.click()

Now that you have selected the competitors you would like to compare against, it is time to select the data you would like to view.  Run the code in the box below by clicking anywhere in the box and pressing shift+enter.

In [ ]:
driver.implicitly_wait(5)
data = {}
exclusions = set()
dict_adds = set()
raw_data = driver.find_element(By.XPATH, '//*[@id="root"]/div/div/div[3]/div/div[4]/div/a[5]/div/span')
raw_data.click()
driver.implicitly_wait(5)
table = driver.find_element(By.XPATH, '//div[@class="rt-table"]')
rows = table.find_elements(By.XPATH, '//div[@class="rt-tr-group"]')
data = {'Wine': [], 'Vintage': [], 'Format': [], 'Point of Sale': [], 'Price': []}
for row in rows:
    cells = row.find_elements(By.XPATH, './/div[@class="rt-td"]')
    if cells[7].text != '':
        data['Wine'].append(cells[3].text)
        data['Vintage'].append(cells[4].text)
        data['Format'].append(cells[5].text)
        data['Point of Sale'].append(cells[2].text)
        data['Price'].append(cells[7].text)
    else:
        exclusions.add(cells[3].text)

max_length = max(len(data['Wine']), len(data['Vintage']), len(data['Format']), len(data['Point of Sale']), len(data['Price']))
data_padded = {
    'Wine': data['Wine'] + [None] * (max_length - len(data['Wine'])),
    'Vintage': data['Vintage'] + [None] * (max_length - len(data['Vintage'])),
    'Format': data['Format'] + [None] * (max_length - len(data['Format'])),
    'Point of Sale': data['Point of Sale'] + [None] * (max_length - len(data['Point of Sale'])),
    'Price': data['Price'] + [None] * (max_length - len(data['Price']))
}

df = pd.DataFrame(data_padded)
df.insert(0, 'Market', list(MARKETS)[int(mkt) - 1])
df.reset_index(drop=True, inplace=True)
df.index += 1
df['Guidance Price'] = 0
df['Markup'] = ''
for index, row in df.iterrows():
    if row['Format'] != None:
        wine_format = row['Format']
        guidance_dict_name = 'guidance' + wine_format.replace(' ', '')
        if hasattr(helperFunctions, guidance_dict_name):
            guidance_dict = getattr(helperFunctions, guidance_dict_name)
            wine_name = row['Wine']
            if wine_name in guidance_dict:
                guidance_price_str = guidance_dict[wine_name]
                guidance_price_match = re.search(r'\b(\d{1,3}(?:,?\d{3})*(?:\.\d{2})?)\b', guidance_price_str)
                if guidance_price_match:
                    guidance_price = float(guidance_price_match.group(1))
                    df.at[index, 'Guidance Price'] = guidance_price_str
                    price_str = row['Price']
                    modified_price = re.sub(r'\s+\$US$', '', price_str)
                    if ',' in price_str:
                        price_parts = modified_price.split(',')
                        if len(price_parts) == 2 and len(price_parts[1]) == 1:
                            modified_price += '0'
                    modified_price = re.sub(r'[^0-9,]', '', modified_price)
                    modified_price = float(modified_price.replace(',', '.'))
                    if guidance_price > modified_price:
                        df.at[index, 'Markup'] = '!!!!!'
                    else:
                        markup = round(modified_price / guidance_price, 2)
                        markup = str(markup) + 'x'
                        df.at[index, 'Markup'] = markup
                    modified_price = '$' + str(modified_price)
                    df.at[index, 'Price'] = modified_price
            else:
                dict_adds.add(wine_name)
if len(dict_adds) > 0:
    print("\nThe following wines are not currently in a guidance dictionary.  Please add them to the proper dictionary in the helperFunctions file, if necessary.")
    for index, wine in enumerate(dict_adds):
        print(f'{index + 1}. {wine}')
if df.empty:
    print("\nThere is no data available for the given criteria.")
else:
    print()
    display(df)
    file_path = f'{list(MARKETS)[int(mkt) - 1]}_data.xlsx'
    df.to_excel(file_path, index_label='Index')
if len(exclusions) > 0:
    print(f"\nThe following wines do not have an average price in {list(MARKETS)[int(mkt) - 1]} so are excluded from the table:")
    for index, name in enumerate(exclusions):
        print(f'{index+1}. {name}')
driver.quit()

If there is data displayed above, there is also an Excel file called: '{name of your market}_data.xlsx' saved to your computer wherever this program is saved to.

Open the file to view the above table in Excel, or use to upload to Tableau.

To run the program again, select Clear All Outputs from the top of the screen and return above to Step 7.